In [4]:
import python_pachyderm

client = python_pachyderm.Client()
print(client.get_remote_version())
print(list(client.list_repo()))

major: 2

[repo {
  name: "histogram"
  type: "user"
}
created {
  seconds: 1639197397
  nanos: 769984000
}
size_bytes_upper_bound: 28998
description: "Output repo for pipeline histogram."
branches {
  repo {
    name: "histogram"
    type: "user"
  }
  name: "master"
}
auth_info {
  permissions: CLUSTER_CREATE_SECRET
  permissions: CLUSTER_LIST_SECRETS
  permissions: SECRET_INSPECT
  permissions: SECRET_DELETE
  permissions: CLUSTER_AUTH_GET_ROBOT_TOKEN
  permissions: REPO_READ
  permissions: REPO_INSPECT_COMMIT
  permissions: REPO_LIST_COMMIT
  permissions: REPO_LIST_BRANCH
  permissions: REPO_LIST_FILE
  permissions: REPO_INSPECT_FILE
  permissions: REPO_ADD_PIPELINE_READER
  permissions: REPO_REMOVE_PIPELINE_READER
  permissions: PIPELINE_LIST_JOB
  permissions: REPO_WRITE
  permissions: REPO_DELETE_COMMIT
  permissions: REPO_CREATE_BRANCH
  permissions: REPO_DELETE_BRANCH
  permissions: REPO_ADD_PIPELINE_WRITER
  permissions: REPO_MODIFY_BINDINGS
  permissions: REPO_DELETE
  permi

In [5]:
client.create_repo("data")
print(list(client.list_repo()))

[repo {
  name: "data"
  type: "user"
}
created {
  seconds: 1639198636
  nanos: 601102000
}
auth_info {
  permissions: CLUSTER_DEBUG_DUMP
  permissions: CLUSTER_GET_PACHD_LOGS
  permissions: CLUSTER_CREATE_SECRET
  permissions: CLUSTER_LIST_SECRETS
  permissions: SECRET_INSPECT
  permissions: SECRET_DELETE
  permissions: CLUSTER_AUTH_GET_ROBOT_TOKEN
  permissions: REPO_READ
  permissions: REPO_INSPECT_COMMIT
  permissions: REPO_LIST_COMMIT
  permissions: REPO_LIST_BRANCH
  permissions: REPO_LIST_FILE
  permissions: REPO_INSPECT_FILE
  permissions: REPO_ADD_PIPELINE_READER
  permissions: REPO_REMOVE_PIPELINE_READER
  permissions: PIPELINE_LIST_JOB
  permissions: REPO_WRITE
  permissions: REPO_DELETE_COMMIT
  permissions: REPO_CREATE_BRANCH
  permissions: REPO_DELETE_BRANCH
  permissions: REPO_ADD_PIPELINE_WRITER
  permissions: REPO_MODIFY_BINDINGS
  permissions: REPO_DELETE
  roles: "repoOwner"
}
, repo {
  name: "histogram"
  type: "user"
}
created {
  seconds: 1639197397
  nanos: 769

In [6]:
with client.commit('data', 'master') as i:
    client.put_file_url(i, 'total_vaccinations_dec_2020_24-31.csv', 'https://raw.githubusercontent.com/PacktPublishing/Reproducible-Data-Science-with-Pachyderm/main/Chapter11-Using-Pachyderm-Notebooks/total_vaccinations_dec_2020_24-31.csv')
print(list(client.list_file(("data", "master"), "")))

[file {
  commit {
    branch {
      repo {
        name: "data"
        type: "user"
      }
      name: "master"
    }
    id: "b4ab87d9c1bc4277bf2a74469070d2c6"
  }
  path: "/total_vaccinations_dec_2020_24-31.csv"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1639198642
  nanos: 775745000
}
size_bytes: 2694
hash: "\252\216m\370\0314\343\004\203j;\332t\275E\235c\312\232\315~+\020\375\236lUp\277\325\345\203"
]


In [7]:
import pandas as pd
pd.read_csv(client.get_file(("data", "master"), "total_vaccinations_dec_2020_24-31.csv"))

,location,date,vaccine,total_vaccinations
0,Chile,2020-12-24,Pfizer/BioNTech,420
1,Chile,2020-12-25,Pfizer/BioNTech,5198
2,Chile,2020-12-26,Pfizer/BioNTech,8338
3,Chile,2020-12-27,Pfizer/BioNTech,8649
4,Czechia,2020-12-27,Pfizer/BioNTech,1252
...,...,...,...,...
63,Iceland,2020-12-31,Oxford/AstraZeneca,0
64,Iceland,2020-12-31,Pfizer/BioNTech,4940
65,Italy,2020-12-31,Pfizer/BioNTech,39826
66,Lithuania,2020-12-31,Pfizer/BioNTech,9741


In [7]:
import pandas as pd

df = pd.read_csv(client.get_file(("data", "master"), "total_vaccinations_dec_2020_24-31.csv"))
data_top = df.head() 

print(data_top)


  location        date          vaccine  total_vaccinations
0    Chile  2020-12-24  Pfizer/BioNTech                 420
1    Chile  2020-12-25  Pfizer/BioNTech                5198
2    Chile  2020-12-26  Pfizer/BioNTech                8338
3    Chile  2020-12-27  Pfizer/BioNTech                8649
4  Czechia  2020-12-27  Pfizer/BioNTech                1252


In [12]:
from python_pachyderm.service import pps_proto
client.create_pipeline(
     pipeline_name="find-vaccinations",
     transform=pps_proto.Transform(
         cmd=["python3"],
         stdin=[
             "import pandas as pd",
             "df = pd.read_csv('/pfs/data/total_vaccinations_dec_2020_24-31.csv')",
             "max_vac = df['total_vaccinations'].idxmax()",
             "row = df.iloc[[max_vac]]",
             "row.to_csv('/pfs/out/max_vaccinations.csv', header=None, index=None, sep=' ', mode='a')",
         ],
         image="amancevice/pandas",
     ),
     input=pps_proto.Input(
         pfs=pps_proto.PFSInput(glob="/", repo="data")
     ),
)
print(list(client.list_pipeline()))



[pipeline {
  name: "find-vaccinations"
}
version: 1
spec_commit {
  branch {
    repo {
      name: "find-vaccinations"
      type: "spec"
    }
    name: "master"
  }
  id: "068be097212f45edb6a8a426e21e12ca"
}
state: PIPELINE_STARTING
job_counts {
  key: 1
  value: 1
}
last_job_state: JOB_CREATED
parallelism: 1
type: PIPELINE_TYPE_TRANSFORM
details {
  transform {
    image: "amancevice/pandas"
    cmd: "python3"
    stdin: "import pandas as pd"
    stdin: "df = pd.read_csv(\'/pfs/data/total_vaccinations_dec_2020_24-31.csv\')"
    stdin: "max_vac = df[\'total_vaccinations\'].idxmax()"
    stdin: "row = df.iloc[[max_vac]]"
    stdin: "row.to_csv(\'/pfs/out/max_vaccinations.csv\', header=None, index=None, sep=\' \', mode=\'a\')"
  }
  created_at {
    seconds: 1639186437
    nanos: 377870828
  }
  output_branch: "master"
  input {
    pfs {
      name: "data"
      repo: "data"
      branch: "master"
      glob: "/"
      repo_type: "user"
    }
  }
  salt: "6271f1e9a95d48ba8bc7ff2827e

In [14]:
client.get_file(("find-vaccinations", "master"), "/max_vaccinations.csv").read()

b'Germany 2020-12-31 Pfizer/BioNTech 206443\n'

In [18]:
client.delete_repo("data", force=True)
client.delete_pipeline("find-vaccinations")

print(list(client.list_repo()))
print(list(client.list_pipeline()))

[]
[]
